In [1]:
import warnings; warnings.filterwarnings('ignore')

In [2]:
from collections import defaultdict

In [3]:
from rbc.heavydb import RemoteHeavyDB

In [4]:
# NBVAL_IGNORE_OUTPUT
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

In [5]:
heavydb.sql_execute('drop table if exists mytable')
heavydb.sql_execute('create table if not exists mytable (x FLOAT, y DOUBLE, i INT, j BIGINT);');
data = defaultdict(list)
for i in range(5):
    data['x'].append(i)
    data['y'].append(i)
    data['i'].append(i)
    data['j'].append(i)
heavydb.load_table_columnar('mytable', **data)

In [6]:
# Overloading UDFs
@heavydb('f32(f32)', 'f64(f64)')
def incr(v):
    return v + 1
@heavydb('i32(i32)', 'i64(i64)')
def incr(v):
    return v + 10

In [7]:
descr, result = heavydb.sql_execute(
    'select incr(x), incr(incr(y)), incr(i), incr(incr(j))'
    'from mytable')

list(result)

[(1.0, 2.0, 10, 20),
 (2.0, 3.0, 11, 21),
 (3.0, 4.0, 12, 22),
 (4.0, 5.0, 13, 23),
 (5.0, 6.0, 14, 24)]